In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q -U  accelerate bitsandbytes


In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "unsloth/Qwen2.5-Coder-32B-Instruct-bnb-4bit"

# Load model chia đều lên 2 GPU T4 (mỗi cái 16GB VRAM)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto" 
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

prompt = "Viết code Python để vẽ biểu đồ hình sin bằng matplotlib."
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=2048)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
!pip install -q -U pynvml # Để đo VRAM

In [ ]:
# ======================================================================
# BƯỚC 1: CÀI ĐẶT VÀ SỬA LỖI XUNG ĐỘT PHIÊN BẢN
# ======================================================================
import os
import subprocess
import sys
import torch
import gc

def install_dependencies():
    # Kiểm tra xem thư viện đã được cài đặt chưa
    try:
        import awq
        import transformers
        # Kiểm tra phiên bản transformers để tránh lỗi PytorchGELUTanh
        from transformers.activations import PytorchGELUTanh
    except (ImportError, NameError, AttributeError):
        print("⏳ Đang cài đặt thư viện (có thể mất 1-2 phút)...")
        # Ép phiên bản transformers cũ hơn để tương thích với AutoAWQ
        packages = [
            "transformers<4.45.0", 
            "autoawq", 
            "optimum", 
            "accelerate"
        ]
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + packages)
        print("✅ Cài đặt hoàn tất. Vui lòng 'Restart Session' nếu đây là lần đầu chạy!")

install_dependencies()

# ======================================================================
# BƯỚC 2: DỌN DẸP BỘ NHỚ (FIX LỖI TRÀN VRAM KHI CHẠY LẠI)
# ======================================================================
def clear_gpu_memory():
    print("🧹 Đang dọn dẹp bộ nhớ GPU để tránh lỗi nạp chồng...")
    global model, tokenizer
    
    # Xóa các biến global nếu tồn tại
    if 'model' in globals():
        del model
    if 'tokenizer' in globals():
        del tokenizer
        
    # Giải phóng bộ nhớ đệm
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize() # Đợi GPU dọn dẹp xong hoàn toàn
    print("✨ Bộ nhớ sạch sẽ. Bắt đầu nạp model mới.")

clear_gpu_memory()

# ======================================================================
# BƯỚC 3: CẤU HÌNH VÀ LOAD MODEL QWEN 2.5 32B AWQ
# ======================================================================
import warnings
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

# Ẩn các cảnh báo rác
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

model_id = "Qwen/Qwen2.5-32B-Instruct-AWQ"

print(f"\n🚀 Đang nạp model: {model_id}")
print("📦 Cấu hình: 2x T4 GPU (32GB VRAM tổng cộng) | 4-bit AWQ")

try:
    # 1. Tải Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    
    # 2. Tải Model với thiết lập tối ưu cho Kaggle
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",           # Tự động chia layer sang 2 GPU T4
        trust_remote_code=True,
        low_cpu_mem_usage=True,      # Giảm tải RAM hệ thống khi nạp
        torch_dtype=torch.float16    # Sử dụng FP16 để tiết kiệm VRAM
    )
    
    # 3. Khởi tạo Streamer để in chữ trực tiếp
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    print("✅ Model đã sẵn sàng trên GPU!")

    # ======================================================================
    # BƯỚC 4: TẠO VĂN BẢN VỚI CHẾ ĐỘ STREAMING
    # ======================================================================
    prompt = "Why is the sky blue?"
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    
    # Chuẩn bị input theo format của Qwen
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to("cuda")

    print(f"\n🤖 Qwen đang trả lời (Streaming):\n{'-'*30}")
    
    # Chạy generate với streamer (kết quả sẽ in ra màn hình ngay lập tức)
    _ = model.generate(
        **model_inputs, 
        streamer=streamer,           # Kích hoạt streaming
        max_new_tokens=1024,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        repetition_penalty=1.1
    )
    
    print(f"{'-'*30}\n✅ Đã hoàn thành phản hồi.")

except Exception as e:
    print(f"\n❌ LỖI: {e}")
    if "PytorchGELUTanh" in str(e):
        print("💡 GỢI Ý: Bạn cần nhấn 'Restart Session' ở menu bên phải rồi chạy lại ô này.")
    elif "CUDA out of memory" in str(e):
        print("💡 GỢI Ý: VRAM đã đầy. Hãy chọn 'Restart Session' để giải phóng hoàn toàn.")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import gc
import pynvml
import time

# Khởi tạo thư viện đo VRAM
pynvml.nvmlInit()

def print_gpu_utilization():
    device_count = pynvml.nvmlDeviceGetCount()
    print(f"--- GPU Memory Status ({device_count} GPUs) ---")
    for i in range(device_count):
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        print(f"GPU {i}: Used {info.used // 1024**2} MB / Total {info.total // 1024**2} MB")
    print("---------------------------------------")

def run_model_test(model_id, use_4bit=True):
    print(f"\n{'='*20} TESTING: {model_id} {'='*20}")
    print(f"Mode: {'4-bit Quantization' if use_4bit else 'Full/Half Precision'}")
    
    # 1. Cấu hình Quantization (Nếu dùng 4-bit để test model to)
    bnb_config = None
    if use_4bit:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )

    try:
        # 2. Load Tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        
        # 3. Load Model với device_map="auto" (Tự động chia qua 2 T4)
        print("Loading model... (This may take time)")
        start_time = time.time()
        
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map="auto", # Tự động rải model lên GPU 0 và GPU 1
            trust_remote_code=True,
            torch_dtype=torch.float16
        )
        print(f"Model loaded in {time.time() - start_time:.2f} seconds.")
        
        # 4. Kiểm tra VRAM sau khi load
        print_gpu_utilization()
        print(f"Model Footprint: {model.get_memory_footprint() // 1024**2} MB")

        # 5. Test Inference (Chạy thử)
        input_text = "Explain quantum physics in one sentence."
        inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
        
        print("\nGenerating response...")
        outputs = model.generate(**inputs, max_new_tokens=2048)
        print(f"Response: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")
        
    except Exception as e:
        print(f"\n❌ ERROR loading {model_id}: {e}")
        # Nếu lỗi Out Of Memory
        if "CUDA out of memory" in str(e):
            print(">>> MODEL IS TOO BIG FOR 2x T4 (32GB Total) <<<")

    finally:
        # 6. Dọn dẹp bộ nhớ (Quan trọng để test model tiếp theo)
        print("\nCleaning up memory...")
        try:
            del model
            del tokenizer
        except:
            pass
        gc.collect()
        torch.cuda.empty_cache()
        print_gpu_utilization()
        print(f"{'='*50}\n")

In [ ]:
# 2. Mức Trung bình (Thử thách nhẹ)
# Qwen 2.5 14B (Alibaba - Model rất mạnh)
run_model_test("Qwen/Qwen2.5-14B-Instruct", use_4bit=True)

In [ ]:
from huggingface_hub import login

# Thay thế bằng token bạn vừa copy ở Bước 2
hf_token = "" 

login(token=hf_token)

# --- Sau đó mới chạy lệnh test ---
run_model_test("google/gemma-2-27b-it", use_4bit=True)

In [ ]:
run_model_test("meta-llama/Llama-3.1-8B-Instruct", use_4bit=True)